# Module 4: Model Context Protocol (MCP) Foundations Lab

## 🎯 Learning Objectives
By the end of this lab, you will:
1. **Understand MCP Architecture** - The three-layer system: Host, Client, Server
2. **Transform Tool Integration** - From direct function calls to standardized protocol
3. **Build MCP Client** - Hands-on development using MCP Python SDK
4. **Integrate with Agents** - Adapt CourseAssistantAgent to use MCP protocol
5. **Experience Protocol Benefits** - Modularity, standardization, and process isolation

## 🏗️ The Transformation Journey

**Today's Mission:** Transform our direct tool integration into a standardized protocol approach!

```
BEFORE: Direct Tool Integration
┌─────────────────┐
│ CourseAssistant │──► search_content_tool()
│ Agent           │
└─────────────────┘

AFTER: MCP Protocol Integration  
┌─────────────────┐    ┌──────────┐    ┌──────────┐
│ CourseAssistant │◄──►│   MCP    │◄──►│   MCP    │
│ Agent (Host)    │    │ Client   │    │ Server   │
└─────────────────┘    └──────────┘    └──────────┘
                                            │
                                       search_content_tool()
```

## 🧠 The "Universal USB" Analogy

Think of MCP as **"Universal USB for AI Tools"**:
- **Before USB**: Every device needed custom cables and drivers
- **After USB**: Standardized interface, plug-and-play compatibility
- **Before MCP**: Every AI app integrates tools differently
- **After MCP**: Standardized protocol, tools work with any MCP-compatible app

## ⏱️ Lab Timeline (60 minutes)
- **Section 1**: Setup & Understanding MCP (10 min)
- **Section 2**: Server Overview & Startup (10 min) 
- **Section 3**: Building MCP Client (20 min)
- **Section 4**: Host Integration (15 min)
- **Section 5**: Testing & Benefits Reflection (5 min)

---

# Section 1: Setup & Understanding MCP (10 minutes)

Let's set up our environment and understand the MCP mental model before diving into the implementation!

**[!NOTE]**
> This notebook is designed to run in the `ai-education` Conda environment.
> - If you have not already, open a terminal and run:
>   ```
>   conda env create -f environment.yml
>   conda activate ai-education
>   ````
>   (Optional) Run the command below to register the environment as a Jupyter kernel and then select the "Python (ai-education)" kernel from the Jupyter kernel menu. 
>   ```
>   python -m ipykernel install --user --name ai-education --display-name "Python (ai-education)"
>   ```

**AWS Credentials Setup:**
- Set your credentials as environment variables in a cell (do NOT share credentials) unless you are using **AWS SageMaker** then Credentials are pre-configured in your environment. No action needed unless you want to override the default region.

In [1]:
# MyBinder users: set your credentials here (do NOT share real keys)
import os
# os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_ACCESS_KEY'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_SECRET_KEY'
# os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'  # or your region

In [7]:
# Import all required libraries
import json
import subprocess
import time
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional

# MCP imports
from mcp import ClientSession
from mcp.client.stdio import stdio_client, StdioServerParameters

# Course content dependencies (same as agents lab)
import boto3
import numpy as np

print("📚 Libraries imported successfully!")
print("🕐 Lab start time:", datetime.now().strftime("%H:%M:%S"))

📚 Libraries imported successfully!
🕐 Lab start time: 01:19:53


## 🔧 Configuration & Course Content Setup

Let's reuse the same course embeddings from the agents lab and set up our AWS configuration:

In [19]:
# Configuration (same as agents lab)
AWS_REGION = "us-west-2"
EMBEDDINGS_FILE = "../embeddings/course_embeddings.json"
EMBEDDING_MODEL = "amazon.titan-embed-text-v2:0"
LLM_MODEL = "anthropic.claude-3-5-sonnet-20241022-v2:0"
MCP_SERVER_PATH = "../scripts/mcp_course_content_server.py"

print(f"🌎 AWS Region: {AWS_REGION}")
print(f"📁 Embeddings file: {EMBEDDINGS_FILE}")
print(f"🧠 LLM Model: {LLM_MODEL}")

# Initialize AWS Bedrock client
try:
    bedrock_client = boto3.client("bedrock-runtime", region_name=AWS_REGION)
    print("✅ Connected to AWS Bedrock successfully!")
except Exception as e:
    print(f"❌ Failed to connect to AWS Bedrock: {e}")
    print("Please ensure your AWS credentials are configured correctly")

🌎 AWS Region: us-west-2
📁 Embeddings file: ../embeddings/course_embeddings.json
🧠 LLM Model: anthropic.claude-3-5-sonnet-20241022-v2:0
✅ Connected to AWS Bedrock successfully!


## 📊 Understanding Course Content (Quick Review)

Let's quickly verify our course embeddings are available - same data we used in the agents lab:

In [9]:
def load_and_verify_embeddings(file_path: str) -> Dict[str, Any]:
    """Load and verify course embeddings are available"""
    try:
        if not Path(file_path).exists():
            print(f"❌ Embeddings file not found: {file_path}")
            print("Please ensure you have the course embeddings from the agents lab")
            return {}
        
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print(f"✅ Loaded embeddings from {file_path}")
        return data
    except Exception as e:
        print(f"❌ Error loading embeddings: {e}")
        return {}

# Load and verify embeddings
embeddings_data = load_and_verify_embeddings(EMBEDDINGS_FILE)

if embeddings_data:
    metadata = embeddings_data['metadata']
    chunks = embeddings_data['chunks']
    
    print(f"\n📈 Content Statistics:")
    print(f"   📄 Files processed: {len(metadata['processed_files'])}")
    print(f"   📝 Content chunks: {metadata['chunk_count']}")
    print(f"   📊 Total words: {metadata['total_words']:,}")
    print(f"   🧠 Embedding dimension: {metadata['embedding_dimension']}")
    
    print(f"\n🎯 Ready for MCP transformation!")
else:
    print("\n⚠️ Cannot proceed without course embeddings")

✅ Loaded embeddings from ../embeddings/course_embeddings.json

📈 Content Statistics:
   📄 Files processed: 6
   📝 Content chunks: 43
   📊 Total words: 18,800
   🧠 Embedding dimension: 1024

🎯 Ready for MCP transformation!


## 🧠 MCP Mental Model: The Three-Layer Architecture

Before we start building, let's understand the key roles in MCP:

### **🏠 Host (Your AI Application)**
- **Role**: The orchestrator - decides what to do and when
- **Example**: CourseAssistantAgent that needs to search content
- **Responsibility**: Business logic, user interaction, tool orchestration

### **📡 Client (Protocol Handler)**  
- **Role**: The translator - converts between host requests and protocol messages
- **Example**: MCP client that handles JSON-RPC communication
- **Responsibility**: Protocol handling, connection management, error translation

### **🔧 Server (Tool Provider)**
- **Role**: The worker - provides actual functionality through standardized interface
- **Example**: Server that exposes course content search as an MCP tool
- **Responsibility**: Tool implementation, input validation, result formatting

### **🔍 Why This Separation Matters:**
- **Modularity**: Each component can be developed and deployed independently
- **Standardization**: Tools work with any MCP-compatible application
- **Security**: Clear boundaries between application logic and tool execution
- **Scalability**: Servers can run anywhere (local process, remote service, containers)

**🎉 Section 1 Complete!**

You now have:
- ✅ MCP Python SDK installed and ready
- ✅ Course embeddings loaded and verified
- ✅ Clear understanding of MCP's three-layer architecture
- ✅ Mental model of protocol benefits vs direct integration

---

# Section 2: Server Overview & Startup (10 minutes)

Now let's examine our pre-built MCP server and get it running! This server transforms our course content search into a standardized MCP tool.

## 📄 The Pre-Built Server

We've created a complete `course_content_server.py` that you'll find in your lab files. Let's examine its key components:

## 🔍 Key Server Components Explained

Let's understand the important parts of our MCP server:

### **1. FastMCP Server Creation**
```python
from mcp.server.fastmcp import FastMCP
mcp = FastMCP("course-content-server")
```
- Creates a standardized MCP server with automatic protocol handling
- Server name identifies it in MCP communications

### **2. Tool Definition with Decorators** 
```python
@mcp.tool()
def search_content(query: str, max_results: int = 3) -> str:
    """Tool description for LLMs"""
    # Implementation here
```
- `@mcp.tool()` decorator automatically exposes function as MCP tool
- Type hints become JSON schema for validation
- Docstring becomes tool description for LLMs

### **3. Process Isolation**
- Server runs as separate Python process
- Communicates via stdio (stdin/stdout)
- Clear security and resource boundaries

### **4. Same Search Logic**
- Reuses identical FAISS and embedding code from agents lab
- Same AWS Bedrock integration
- Same course content data

**The key insight:** We've wrapped our existing search functionality in a standardized protocol interface!

## 🚀 Server Startup Test

Let's start our MCP server and verify it initializes correctly:

In [20]:
def test_server_startup():
    """Test that our MCP server starts up correctly"""
    print("🚀 Testing MCP server startup...")
    
    try:
        # Get absolute path to the server file
        server_path = os.path.abspath(MCP_SERVER_PATH)
        
        # Start server process
        print("📍 Starting server process...")
        server_process = subprocess.Popen(
            ["python", server_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Give it time to initialize
        print("⏳ Waiting for server initialization...")
        time.sleep(5)
        
        # Check if process is still running (didn't crash)
        if server_process.poll() is None:
            print("✅ Server started successfully and is running")
            print("🔍 Server process ID:", server_process.pid)
            
            # Terminate the server
            server_process.terminate()
            server_process.wait(timeout=5)
            print("🛑 Server stopped cleanly")
            
            return True
        else:
            # Server crashed
            stdout, stderr = server_process.communicate()
            print("❌ Server failed to start")
            print("STDOUT:", stdout[:500] if stdout else "None")
            print("STDERR:", stderr[:500] if stderr else "None")
            return False
            
    except Exception as e:
        print(f"❌ Error testing server startup: {e}")
        
        # Clean up any remaining process
        try:
            if 'server_process' in locals():
                server_process.terminate()
        except:
            pass
        
        return False

# Run the test
startup_success = test_server_startup()

if startup_success:
    print("\n🎯 Server test successful! Ready to build MCP client.")
else:
    print("\n⚠️ Server test failed. Please check AWS credentials and embeddings file.")

🚀 Testing MCP server startup...
📍 Starting server process...
⏳ Waiting for server initialization...
✅ Server started successfully and is running
🔍 Server process ID: 68020
🛑 Server stopped cleanly

🎯 Server test successful! Ready to build MCP client.


## 💡 What Just Happened?

We've successfully demonstrated:

1. **Process Isolation**: The server runs as a separate Python process
2. **Standardized Interface**: FastMCP handles all the protocol details
3. **Tool Exposure**: Our search function is now available as an MCP tool
4. **Same Functionality**: Identical search capabilities as the agents lab

**Key Insight**: The server transforms our function into a network service that speaks a standardized protocol!

**🎉 Section 2 Complete!**

You now have:
- ✅ Understanding of MCP server architecture using FastMCP
- ✅ Working server that exposes course content search as standardized tool
- ✅ Verification that server starts up and initializes correctly
- ✅ Foundation ready for client development

---

# Section 3: Building MCP Client (20 minutes)

Now for the main event - building our MCP client! This is where you'll gain hands-on experience with the protocol communication patterns.

## 🎯 Client Role Recap

The MCP client acts as a **protocol translator**:
- **Converts** host requests into MCP protocol messages
- **Manages** connection lifecycle with the server process  
- **Handles** async communication via stdio transport
- **Translates** MCP responses back to host-friendly format

## 🔧 Client Implementation Strategy

We'll build a client class that:
1. **Connects** to our server via stdio transport
2. **Discovers** available tools through capability negotiation
3. **Invokes** tools using the MCP protocol
4. **Handles** errors and connection management

Let's start building!

In [21]:
class CourseContentMCPClient:
    """
    MCP Client for course content search
    
    This client demonstrates the key patterns for MCP communication:
    - Server connection via stdio transport
    - Tool discovery and capability negotiation
    - Async tool invocation
    - Error handling and connection management
    """
    
    def __init__(self, server_script_path: str = MCP_SERVER_PATH):
        """
        Initialize the MCP client
        
        Args:
            server_script_path: Path to the MCP server script
        """
        self.server_params = StdioServerParameters(
            command="python",
            args=[server_script_path]
        )
        self.available_tools = []
        print(f"📡 MCP Client initialized for server: {server_script_path}")
    
    async def discover_tools(self) -> List[str]:
        """
        Discover available tools from the server
        
        This demonstrates the MCP capability negotiation process.
        
        Returns:
            List of available tool names
        """
        try:
            print("🔍 Discovering available tools from server...")
            
            # Connect to server via stdio transport
            async with stdio_client(self.server_params) as (read_stream, write_stream):
                async with ClientSession(read_stream, write_stream) as session:
                    # Initialize the connection
                    await session.initialize()
                    print("✅ Connection established with server")
                    
                    # List available tools
                    tools_response = await session.list_tools()
                    
                    # Extract tool names
                    tool_names = [tool.name for tool in tools_response.tools]
                    self.available_tools = tool_names
                    
                    print(f"🎯 Found {len(tool_names)} available tools:")
                    for tool in tools_response.tools:
                        print(f"   - {tool.name}: {tool.description}")
                    
                    return tool_names
        
        except Exception as e:
            print(f"❌ Error discovering tools: {e}")
            return []
    
    async def search_content(self, query: str, max_results: int = 3) -> str:
        """
        Search course content using the MCP search_content tool
        
        This demonstrates the core MCP tool invocation pattern.
        
        Args:
            query: Search query
            max_results: Maximum number of results
            
        Returns:
            Search results as formatted string
        """
        try:
            print(f"🔍 Searching for: '{query[:50]}{'...' if len(query) > 50 else ''}'")
            
            # Connect to server
            async with stdio_client(self.server_params) as (read_stream, write_stream):
                async with ClientSession(read_stream, write_stream) as session:
                    # Initialize connection
                    await session.initialize()
                    
                    # Call the search_content tool
                    result = await session.call_tool(
                        "search_content",
                        {
                            "query": query,
                            "max_results": max_results
                        }
                    )
                    
                    # Extract the content from the result
                    if result.content:
                        # MCP results come wrapped in content objects
                        content = result.content[0].text if result.content else "No content returned"
                        print("✅ Search completed successfully")
                        return content
                    else:
                        print("⚠️ No content in search result")
                        return "No results found"
        
        except Exception as e:
            error_msg = f"❌ Error during search: {e}"
            print(error_msg)
            return error_msg
    
    async def get_server_status(self) -> str:
        """
        Get server status using the MCP get_server_status tool
        
        This demonstrates tool invocation for server health checks.
        
        Returns:
            Server status information
        """
        try:
            print("📊 Checking server status...")
            
            async with stdio_client(self.server_params) as (read_stream, write_stream):
                async with ClientSession(read_stream, write_stream) as session:
                    await session.initialize()
                    
                    result = await session.call_tool("get_server_status", {})
                    
                    if result.content:
                        content = result.content[0].text if result.content else "No status available"
                        return content
                    else:
                        return "No status information available"
        
        except Exception as e:
            error_msg = f"❌ Error checking server status: {e}"
            print(error_msg)
            return error_msg

print("✅ MCP Client class implemented!")
print("🎯 Ready to test client functionality")

✅ MCP Client class implemented!
🎯 Ready to test client functionality


## 🧪 Testing Our MCP Client

Let's test our client by connecting to the server and discovering tools:

In [22]:
# Create client instance
mcp_client = CourseContentMCPClient()

# Test tool discovery
print("🔍 Testing Tool Discovery")
print("=" * 40)

# Run async function in notebook
available_tools = await mcp_client.discover_tools()

if available_tools:
    print(f"\n✅ Successfully discovered {len(available_tools)} tools!")
    print("🎯 Client is ready for tool invocation")
else:
    print("\n❌ Tool discovery failed")
    print("Please ensure the server is working correctly")

📡 MCP Client initialized for server: ../scripts/mcp_course_content_server.py
🔍 Testing Tool Discovery
🔍 Discovering available tools from server...
✅ Connection established with server
🎯 Found 2 available tools:
   - search_content: 
    Search course content using semantic similarity
    
    This tool searches through AI/ML course materials to find relevant content
    based on the provided query. It uses vector embeddings and semantic similarity
    to return the most relevant sections.
    
    Args:
        query: The search query - what you want to find in the course content
        max_results: Maximum number of results to return (default: 3, max: 10)
    
    Returns:
        Formatted search results with titles, sources, and content snippets
    
   - get_server_status: 
    Get the current status of the course content server
    
    Returns information about the server state, including whether it's properly
    initialized and ready to handle search requests.
    
    Returns

## 🎯 Testing Tool Invocation

Now let's test actual tool calls through the MCP protocol:

In [23]:
# Test server status check
print("📊 Testing Server Status Check")
print("=" * 40)

status_result = await mcp_client.get_server_status()
print("Server Status Response:")
print(status_result)

print("\n" + "="*60 + "\n")

# Test content search
print("🔍 Testing Content Search")
print("=" * 40)

test_query = "What are the key differences between agents and LLMs?"
search_result = await mcp_client.search_content(test_query, max_results=2)

print("Search Query:", test_query)
print("\nSearch Results:")
print("-" * 30)
print(search_result[:500] + "..." if len(search_result) > 500 else search_result)

📊 Testing Server Status Check
📊 Checking server status...
Server Status Response:
✅ Server Status: READY
📊 Content chunks loaded: 43
🌎 AWS Region: us-west-2
🧠 Embedding Model: amazon.titan-embed-text-v2:0
🔍 Ready to handle search requests


🔍 Testing Content Search
🔍 Searching for: 'What are the key differences between agents and LL...'
✅ Search completed successfully
Search Query: What are the key differences between agents and LLMs?

Search Results:
------------------------------
Found 2 relevant content sections for 'What are the key differences between agents and LLMs?':

1. **From LLMs to Agents: Why Go Further?** (Relevance: 0.628)
   Source: agents.html
   Content: ## From LLMs to Agents: Why Go Further?

While LLMs are incredibly versatile, many real-world applications require more than just language understanding. This is where LLM-powered agents come in.

🤖 **Agentic LLM application** is a software system that wraps around the LLM, operating in a loop—observing i...


## 🎓 Understanding the MCP Communication Flow

Let's examine what just happened:

### **1. Connection Establishment**
```python
async with stdio_client(server_params) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
```
- **stdio_client**: Creates subprocess and connects via stdin/stdout
- **ClientSession**: Manages MCP protocol lifecycle

### **2. Capability Negotiation**  
```python
await session.initialize()
tools_response = await session.list_tools()
```
- **initialize()**: Handshake between client and server
- **list_tools()**: Discover what tools the server provides

### **3. Tool Invocation**
```python
result = await session.call_tool("search_content", {"query": query})
```
- **call_tool()**: JSON-RPC call with tool name and arguments
- **result.content**: Server response wrapped in MCP format

### **🔑 Key Differences from Direct Tool Calls:**
- **Async**: All operations are asynchronous (vs sync function calls)
- **Protocol**: Structured JSON-RPC communication (vs Python function calls)
- **Process Boundary**: Client and server in separate processes (vs same process)
- **Discovery**: Tools are discovered at runtime (vs compile-time imports)
- **Standardization**: Any MCP client can use any MCP server (vs tight coupling)

**🎉 Section 3 Complete!**

You now have:
- ✅ **Working MCP client** that communicates with server via protocol
- ✅ **Understanding of async patterns** and protocol communication
- ✅ **Experience with tool discovery** and capability negotiation
- ✅ **Hands-on MCP integration** ready for host application
- ✅ **Protocol vs direct integration** comparison and insights

---

# Section 4: Host Integration (15 minutes)

Time for the final transformation! Let's integrate our MCP client with the CourseAssistantAgent and see the complete protocol-based system in action.

## 🎯 The Transformation Goal

We're adapting the CourseAssistantAgent from the agents lab to use MCP protocol instead of direct tool calls:

**Before (Agents Lab):**
```python
search_results = search_content_tool(user_input)  # Direct function call
response = self._call_llm(prompt)  # Sync operation
```

**After (MCP Protocol):**
```python
search_results = await mcp_client.search_content(user_input)  # Protocol call
response = self._call_llm(prompt)  # LLM call stays the same
```

## 🔧 Building the MCP-Enabled Agent

In [25]:
class MCPCourseAssistantAgent:
    """
    CourseAssistantAgent adapted to use MCP protocol
    
    This demonstrates how to transform an agent from direct tool integration
    to standardized MCP protocol integration while maintaining the same
    core functionality and user experience.
    """
    
    def __init__(self, mcp_client: CourseContentMCPClient):
        """
        Initialize agent with MCP client
        
        Args:
            mcp_client: Configured MCP client for tool communication
        """
        self.mcp_client = mcp_client
        
        # AWS Bedrock client for LLM calls (same as agents lab)
        self.bedrock_client = boto3.client("bedrock-runtime", region_name=AWS_REGION)
        
        print("🤖 MCP-enabled Course Assistant Agent initialized!")
        print("   📡 Using MCP protocol for tool communication")
    
    def _call_llm(self, prompt: str, max_tokens: int = 500, temperature: float = 0.1) -> str:
        """
        Helper method to call Claude via Bedrock
        
        This is identical to the agents lab implementation - 
        LLM integration doesn't change when using MCP!
        """
        try:
            request_body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": temperature
            }

            response = self.bedrock_client.invoke_model(
                modelId=LLM_MODEL,
                body=json.dumps(request_body),
                contentType='application/json'
            )
            
            response_body = json.loads(response['body'].read())
            return response_body['content'][0]['text'].strip()
        
        except Exception as e:
            return f"Error calling LLM: {str(e)}"
    
    async def decide_and_act(self, user_input: str) -> str:
        """
        Main agent decision cycle using MCP protocol
        
        Key differences from agents lab version:
        1. Method is now async (MCP requires async)
        2. Tool calls go through MCP client instead of direct functions
        3. Same orchestration logic and LLM integration
        
        Args:
            user_input: The user's question or request
            
        Returns:
            Agent's response
        """
        
        if not user_input.strip():
            return "I'd be happy to help! Please ask me a question about the course content."
        
        # OBSERVE: Analyze the user's input (same as agents lab)
        print(f"🔍 Agent observing: '{user_input}'")
        
        # PLAN: Simple plan - search content then respond (same logic as agents lab)
        print("📋 Agent planning: Will search content via MCP and provide response")
        
        # ACT: Execute the plan
        
        # Step 1: Search for relevant content using MCP protocol
        print("⚡ Agent acting: Searching course content via MCP...")
        try:
            search_results = await self.mcp_client.search_content(user_input, max_results=3)
        except Exception as e:
            search_results = f"Error accessing search tool: {e}"
        
        # Step 2: Generate response using search results (same as agents lab)
        print("⚡ Agent acting: Generating response with found content...")
        
        response_prompt = f"""You are a helpful course assistant for an AI/ML education program.

A student asked: "{user_input}"

Here's relevant content from the course materials:
{search_results}

Based on this content, provide a clear, helpful answer to the student's question. 
Focus on the key concepts and make it educational. If the search results don't 
contain relevant information, say so politely and offer to help with other topics.

Keep your response concise but complete (2-3 paragraphs maximum)."""
        
        main_response = self._call_llm(response_prompt, max_tokens=400)
        
        return main_response

print("✅ MCP-enabled CourseAssistantAgent implemented!")
print("🎯 Ready for testing and comparison")

✅ MCP-enabled CourseAssistantAgent implemented!
🎯 Ready for testing and comparison


## 🧪 Testing the MCP-Enabled Agent

Let's test our MCP-enabled agent and see it in action:

In [26]:
# Create the MCP-enabled agent
mcp_agent = MCPCourseAssistantAgent(mcp_client)

print("🤖 Testing MCP-Enabled Course Assistant Agent\n")

# Test with a course-related question
test_question = "What are the main components of an AI agent?"

print(f"User: {test_question}")
print("\n" + "="*60 + "\n")

# Run the agent (note: await because it's now async)
response = await mcp_agent.decide_and_act(test_question)

print(f"\nAgent Response:")
print("-" * 20)
print(response)

print("\n" + "="*60 + "\n")

🤖 MCP-enabled Course Assistant Agent initialized!
   📡 Using MCP protocol for tool communication
🤖 Testing MCP-Enabled Course Assistant Agent

User: What are the main components of an AI agent?


🔍 Agent observing: 'What are the main components of an AI agent?'
📋 Agent planning: Will search content via MCP and provide response
⚡ Agent acting: Searching course content via MCP...
🔍 Searching for: 'What are the main components of an AI agent?'
✅ Search completed successfully
⚡ Agent acting: Generating response with found content...

Agent Response:
--------------------
I notice that the provided course materials don't directly answer the question about the main components of an AI agent. However, I can provide a clear, foundational answer based on standard AI principles:

An AI agent typically consists of four main components: sensors, actuators, knowledge base, and reasoning engine. Sensors allow the agent to perceive its environment (like a chatbot receiving text input or a robot's came

## 🎓 Real-World Integration Patterns

Let's test a few more scenarios to understand when MCP shines:

In [28]:
# Test multiple questions to see the agent in action
print("🧪 Multiple Question Testing\n")

test_questions = [
    "How does prompt engineering improve AI outputs?",
    "What makes MCP different from direct tool integration?",
    "What are the benefits of using vector embeddings?"
]

for i, question in enumerate(test_questions, 1):
    print(f"🎯 Test {i}: {question}")
    print("-" * 50)
    
    response = await mcp_agent.decide_and_act(question)
    
    # Show first 150 characters of response
    preview = response[:150] + "..." if len(response) > 150 else response
    print(f"Response: {preview}")
    print("\n")

print("✅ All tests completed successfully!")
print("🎯 MCP integration working as expected")

🧪 Multiple Question Testing

🎯 Test 1: How does prompt engineering improve AI outputs?
--------------------------------------------------
🔍 Agent observing: 'How does prompt engineering improve AI outputs?'
📋 Agent planning: Will search content via MCP and provide response
⚡ Agent acting: Searching course content via MCP...
🔍 Searching for: 'How does prompt engineering improve AI outputs?'
✅ Search completed successfully
⚡ Agent acting: Generating response with found content...
Response: Based on the course materials, prompt engineering improves AI outputs by providing better context and instructions to AI models through carefully craf...


🎯 Test 2: What makes MCP different from direct tool integration?
--------------------------------------------------
🔍 Agent observing: 'What makes MCP different from direct tool integration?'
📋 Agent planning: Will search content via MCP and provide response
⚡ Agent acting: Searching course content via MCP...
🔍 Searching for: 'What makes MCP differe

**🎉 Section 4 Complete!**

You now have:
- ✅ **Complete MCP-enabled CourseAssistantAgent** with same functionality as agents lab
- ✅ **Understanding of async agent patterns** and protocol integration
- ✅ **Side-by-side comparison** of direct vs MCP approaches
- ✅ **Real-world integration experience** with working examples
- ✅ **Insight into when to use MCP** vs direct integration

---

# Section 5: Benefits & Production Considerations (5 minutes)


## 🎯 MCP Benefits Demonstrated

Throughout this lab, we've experienced the key benefits of MCP:

### **1. 🔧 Modularity**
- **Server Development**: Independent of client applications
- **Tool Updates**: Server changes don't break existing clients
- **Language Independence**: Server could be Python, client could be JavaScript

### **2. 📐 Standardization** 
- **Universal Interface**: Any MCP client can use any MCP server
- **Tool Discovery**: Runtime discovery vs compile-time dependencies
- **Protocol Evolution**: Versioned protocol for future compatibility

### **3. 🛡️ Security & Isolation**
- **Process Boundaries**: Server runs in separate process with clear resource limits
- **Controlled Communication**: Only structured protocol messages, no direct memory access
- **Error Isolation**: Server crashes don't take down the host application

### **4. 🚀 Scalability**
- **Remote Deployment**: Servers can run anywhere (local, remote, containers)
- **Load Distribution**: Multiple server instances for high availability
- **Transport Flexibility**: stdio (development) → HTTP (production)

## 🚀 Production Considerations

When moving to production, consider these MCP deployment patterns:

### **🌐 Transport Upgrades**
- **Development**: stdio transport (what we used)
- **Production**: HTTP transport with authentication
- **Enterprise**: OAuth 2.1 with proper token management

### **⚖️ Scaling Strategies**
- **Single Server**: Good for development and small deployments
- **Load Balanced**: Multiple server instances behind load balancer
- **Microservices**: Different tools as separate MCP services
- **Container Orchestration**: Kubernetes deployments with auto-scaling

### **🔒 Security Enhancements**
- **Authentication**: API keys or OAuth for server access
- **Authorization**: Role-based access to different tools
- **Rate Limiting**: Prevent abuse of expensive operations
- **Audit Logging**: Track all tool invocations for compliance

### **📈 Monitoring & Observability**
- **Health Checks**: Server status and tool availability
- **Performance Metrics**: Latency, throughput, error rates
- **Tool Usage Analytics**: Which tools are used most frequently
- **Error Tracking**: Protocol errors, tool failures, timeouts

## 🎓 When to Choose MCP vs Direct Integration

**✅ Choose MCP When:**
- Building tools that multiple applications will use
- Need strong security boundaries between app and tools
- Tools are complex, resource-intensive, or stateful
- Team structure benefits from independent development
- Future interoperability is important

**✅ Choose Direct Integration When:**
- Rapid prototyping and simple applications
- Performance is critical and latency matters
- Tools are simple, stateless functions
- Single team developing both app and tools
- Debugging simplicity is paramount

**🎯 Hybrid Approach:**
Many production systems use both - direct integration for core functionality and MCP for extensible tools!

## 🎉 Lab Complete!

**Congratulations!** You've successfully transformed a direct tool integration into a standardized MCP protocol system.

### **🏆 What You've Accomplished:**

**🔧 Technical Skills:**
- ✅ Built MCP client using Python SDK with proper async patterns
- ✅ Integrated MCP protocol into agent orchestration
- ✅ Handled process management, connections, and error scenarios
- ✅ Experienced tool discovery and capability negotiation

**🧠 Conceptual Understanding:**
- ✅ MCP three-layer architecture (Host, Client, Server)
- ✅ Protocol vs direct integration trade-offs
- ✅ Benefits of standardization and modularity
- ✅ Production deployment considerations

**🚀 Real-World Readiness:**
- ✅ Know when to choose MCP vs direct integration
- ✅ Can integrate MCP into existing applications
- ✅ Understand scaling and security patterns
- ✅ Ready to build with protocol standards

### **🌟 The Bigger Picture**

You've experienced the shift from **custom integrations** to **standardized protocols** - the same evolution that transformed computing with standards like USB, HTTP, and TCP/IP.

**MCP represents the future of AI tool integration** - where any tool can work with any application through a common protocol.

**🎯 You're now equipped to build AI applications that leverage the growing ecosystem of MCP-compatible tools and services!**

---

**Thank you for completing the MCP Foundations Lab!** 🚀